In [7]:
import requests
import codecs
import csv
from bs4 import BeautifulSoup

def scrape_events(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        events_data = []

        # Modify the following code based on the structure of the HTML on the website
        event_elements = soup.find_all('div', {'class': 'title'})

        for event_element in event_elements:
            anchor_tag = event_element.find('a')

            # Check if an anchor tag is found before accessing its text
            if anchor_tag:
                event_name = anchor_tag.text.strip()
            else:
                event_name = ""

            # Finding sibling elements for additional information
            date_element = event_element.find_next_sibling('div', {'class': 'subtitle'})
            address_element = event_element.find_next_sibling('div', {'class': 'info'})
            summary_element = event_element.find_next_sibling('div', {'class': 'summary'})

            if date_element:
                event_date = date_element.text.replace('Event Date:', '').strip()
            else:
                event_date = ""

            if address_element:
                event_address = address_element.text.replace('Address:', '').strip()
            else:
                event_address = ""

            if summary_element:
                event_summary = summary_element.text.strip()
            else:
                event_summary = ""

            event_info = {
                'Event Name': event_name,
                'Event Date': event_date,
                'Event Address': event_address,
                'Event Summary': event_summary,
            }

            events_data.append(event_info)

        return events_data
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None

def save_to_csv(data, csv_filename):
    if data:
        keys = data[0].keys()

        with codecs.open(csv_filename, 'w', 'utf-8-sig') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=keys)
            writer.writeheader()
            writer.writerows(data)

if __name__ == "__main__":
    events_url = 'https://mypenang.gov.my/events/all-events/?quarter=4&lg=en'
    events_data = scrape_events(events_url)

    if events_data:
        csv_filename = 'events_data.csv'
        save_to_csv(events_data, csv_filename)
        print(f"Events data saved to {csv_filename}")
    else:
        print("No events data retrieved.")


Events data saved to events_data.csv
